In [1]:
!cp drive/My\ Drive/natural-language-processing/Comments_Data.zip .
!unzip Comments_Data.zip
!cp drive/My\ Drive/natural-language-processing/stopwords-farsi.txt .

Archive:  Comments_Data.zip
  inflating: test_nolabel_comments.csv  
  inflating: train_comments.csv      


### Loading Data
- I am going to read only 10000 rows because of low ram issues

In [2]:
import pandas as pd
pd.options.display.max_colwidth = 100

In [3]:
df = pd.read_csv('train_comments.csv', index_col='id', usecols=['id','title','comment','verification_status'],nrows=10000)
df.head()

,title,comment,verification_status
id,,,
0,نسبت به قیمتش خوبه ولی یه کم کوچیکه,کوچیکه - کلید خاموش و روشنش مسخره اس,1
1,شومیز,خیلی بد فرمه انگار مانتوی بارداریه. اصلا با عکسی که گذاشتن مطابقت نداره,0
2,جنس بسیار ضعیفه,پیشنهادم اینه ک کسی نخره چون پولشو میندازه دور,0
3,رضایت از خرید,طعمش از بقیه کاپوچینو های فوری بهتره,0
4,ارسال جنس تقلبی,برای من چهار عدد ارسال شد که یکی از کارتن ها باز شده بود و شمع تقلبی داخلش گذاشته بودن و با چسب ...,1


In [4]:
df.describe()

,verification_status
count,10000.000000
mean,0.172100
std,0.377486
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


### Data Cleaning
- Make text all lower case (Normalization)
- Punctuation Removal
- Numerical Value Removal
- Stop-words Removal
- Tokenization

In [4]:
df.comment.loc[0]

'کوچیکه - کلید خاموش و روشنش مسخره اس'

In [5]:
import re
import string

def clean_text(text):
  text = text.lower()
  text = re.sub('[.*؟،?\/]', '', text)  # remove .*?
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # remove punctuation
  text = re.sub('\w*\d\w*', '', text) # remove any number surrounded with a string
  return text

df.comment = df.comment.apply(lambda x: clean_text(x))
df.head(5)

,title,comment,verification_status
id,,,
0,نسبت به قیمتش خوبه ولی یه کم کوچیکه,کوچیکه کلید خاموش و روشنش مسخره اس,1
1,شومیز,خیلی بد فرمه انگار مانتوی بارداریه اصلا با عکسی که گذاشتن مطابقت نداره,0
2,جنس بسیار ضعیفه,پیشنهادم اینه ک کسی نخره چون پولشو میندازه دور,0
3,رضایت از خرید,طعمش از بقیه کاپوچینو های فوری بهتره,0
4,ارسال جنس تقلبی,برای من چهار عدد ارسال شد که یکی از کارتن ها باز شده بود و شمع تقلبی داخلش گذاشته بودن و با چسب ...,1


### More Data Cleaning

In [6]:
def clean_text_more(text):
  text = re.sub('[''""...]', '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\r', '', text)
  text = re.sub('[...]','',text).replace('\u200c', '')
  return text

df.comment = df.comment.apply(lambda x: clean_text_more(x))
df['comment'].head(50)

id
0                                                                     کوچیکه  کلید خاموش و روشنش مسخره اس
1                                  خیلی بد فرمه انگار مانتوی بارداریه اصلا با عکسی که گذاشتن مطابقت نداره
2                                                          پیشنهادم اینه ک کسی نخره چون پولشو میندازه دور
3                                                                    طعمش از بقیه کاپوچینو های فوری بهتره
4     برای من چهار عدد ارسال شد که یکی از کارتن ها باز شده بود و شمع تقلبی داخلش گذاشته بودن و با چسب ...
5     به نظر من نباید عکس  با خود جنس مقایرت داشته باشه  میشه هم طوری عکاسی بکنید که  به اشتباه نیفتند...
6     بیش از  ماه پیش  این سیستم را با توجه به نضرات کاربران برای بچه ها که در سالهای آخر دبیرستان هست...
7         میشه سایز  رو دوباره موجود کنیدمن دیر رسیدم به پیشنهاد شگفت انگیزشکاش دوباره شگفت انگیز بزاریدش
8                                                                                               راضی بودم
9                                          

In [7]:
re.sub('[...]','','من این ساعت رو خریداری کردم و بعد از تحویل در کمال شگفتی متوجه شدم که روی جعبه مشخصات ساعت دیگه ای نوشته شده که با ساعتی که ...').replace('\u200c', '')


'من این ساعت رو خریداری کردم و بعد از تحویل در کمال شگفتی متوجه شدم که روی جعبه مشخصات ساعت دیگه ای نوشته شده که با ساعتی که '

### Organizing the data
- Corpus which is already ready
- Document-Term matrix

In [8]:
df.to_pickle('corpus.pkl')

#### Document-Term Matrix

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.externals import joblib ## can use joblib for better performance

### Set a custom persian stop words
stop_words = open('stopwords-farsi.txt').readlines()
stop_words_to_list = [word.strip() for word in stop_words]

cv = TfidfVectorizer(stop_words=stop_words_to_list)
# vec.fit(df.comment)
# joblib.dump(vec, 'vec_count.joblib')
# vec = joblib.load('vec_count.joblib')
# df_cv = vec.transform(df.comment)
df_cv = cv.fit_transform(df.comment)
df_dtm = pd.DataFrame(df_cv.toarray(),columns=cv.get_feature_names())
df_dtm.index = df.index
df_dtm.tail()

,aali,ac,acer,actyon,adata,adataتا,adsl,after,air,airdots,airdotspro,aks,al,ali,alt,amoled,anc,anker,antutu,ar,arc,armor,aspire,asus,auto,aux,avalesh,awlie,az,aتا,bad,bag,band,bar,bb,beats,bekharin,benchmark,bi,bkخریدم,...,یکساعت,یکسال,یکسالزنجیراشم,یکساله,یکسالو,یکسالی,یکسان,یکسانه,یکسره,یکسری,یکش,یکطرفش,یکطرفه,یکم,یکماه,یکماهه,یکماهی,یکمرتبه,یکمش,یکمقدار,یکمم,یکمواسه,یکمی,یکمیل,یکنم,یکنواخت,یکنواخته,یکنواختی,یکنی,یکه,یکهبا,یکهفته,یککیسه,یکیش,یکیشو,یکیشون,یکیه,یگیش,یی,یییییییییییییییییییر
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
